In [1]:
import random
import pandas as pd
import Covid19_model as CM
import numpy as np
import shutil
import sys
import os.path
import math
import csv
import lhsmdu
import time
from Pyomo_solve import solve_ode

# Set parameter

In [2]:
D = 1e2
n_site = 24
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 2.8e6
length = 50

In [3]:
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
seed = 0
env = CM.Env_model(init_state, deter_para, in_deter_para, seed)
env.set_para_truth(para_truth)

In [5]:
tic = time.time()
random.seed(seed)
observation, state = init_state.copy(), init_state.copy()
state_buffer, new_buffer  = np.zeros([T, 6]), np.zeros([T, 4])
score, ni, ne = 0, 0, 0
done = False
index = [0]
start_time = 6
aa = [1] * 5
for t in range(T):
    p_set = env.para_sampling(100)
    para = p_set[index[0]]
    action_buffer = [] 
    reward_buffer = []
    s = np.array(state)
    ac = []
    if t > start_time:
        for a in range(n_site):
            try:
                aa = solve_ode(s[:,a], np.sum(s[1:-1,a])*budget/np.sum(s[1:-1,:]), para, length)
            except Exception:
                print("pyomo fail")
            ac.append(aa)
        action = np.array(ac).T
        if budget < 0: 
            action = np.zeros((5,n_site))
    else:
        action = np.zeros((5,n_site))
        
    
    next_state, obs, reward, cost, new_i, new_e, ck = env.online_state_estimate(state, observation, 
                                                                                action, p_set, 100, 1)
    #if not done and t > start_time: 
     #   env.space_update(ck, p_set, n_select)
    index = ck.argsort()[0][:1]
    score += reward
    budget -= cost
    ni += new_i
    ne += new_e

    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'True State Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, score:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]),
                  budget, score))
    print('\r', 'Observe Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]),
                  np.sum(state[4]), np.sum(state[5]))) 
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time() - tic)

 True State Time: 0 | S: 25495832.0, E: 168.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2800000.00, score:-343.91
 Observe Time: 0 | S: 25495832.0, E: 168.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0
 Time: 0 | new_e:79.4, cum_e:79.4, new_i:70.9, cum_i:70.9
 True State Time: 1 | S: 25495699.7, E: 221.7, A: 0.0, I: 70.9, D: 0.0, R: 7.6, Budget: 2800000.00, score:-831.52
 Observe Time: 1 | S: 25495697.0, E: 214.0, A: 0.0, I: 70.9, D: 0.0, R: 0.0
 Time: 1 | new_e:111.4, cum_e:190.8, new_i:98.1, cum_i:169.0
 True State Time: 2 | S: 25495514.0, E: 299.3, A: 0.0, I: 168.9, D: 0.1, R: 17.7, Budget: 2800000.00, score:-1524.58
 Observe Time: 2 | S: 25495519.0, E: 298.0, A: 0.0, I: 168.9, D: 0.1, R: 0.0
 Time: 2 | new_e:157.4, cum_e:348.2, new_i:137.0, cum_i:305.9
 True State Time: 3 | S: 25495251.9, E: 411.0, A: 0.0, I: 305.7, D: 0.3, R: 31.2, Budget: 2800000.00, score:-2511.29
 Observe Time: 3 | S: 25495267.0, E: 409.0, A: 0.0, I: 305.7, D: 0.3, R: 18.0
 Time: 3 | new_e:223.0, cum_e:571.2, new_i:192.7, cu

 True State Time: 18 | S: 24624022.0, E: 9.0, A: 652.9, I: 1615.7, D: 6.3, R: 869694.1, Budget: 2157736.06, score:-15731.82
 Observe Time: 18 | S: 24624238.0, E: 0.0, A: 652.9, I: 1615.7, D: 6.3, R: 869685.0
 Time: 18 | new_e:4.1, cum_e:3540.2, new_i:73.7, cum_i:4567.1
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 19 | S: 24563991.6, E: 9.0, A: 556.1, I: 1487.6, D: 6.5, R: 929949.3, Budget: 2126367.36, score:-15955.01
 Observe Time: 19 | S: 24564216.0, E: 0.0, A: 556.1, I: 1487.6, D: 6.5, R: 929940.0
 Time: 19 | new_e:46.8, cum_e:3587.0, new_i:91.7, cum_i:4658.8
 True State Time: 20 | S: 24496466.7, E: 51.5, A: 474.1, I: 1413.4, D: 6.9, R: 

 True State Time: 36 | S: 23519068.5, E: 18.1, A: 85.2, I: 377.9, D: 8.8, R: 1976441.4, Budget: 1439696.96, score:-17022.54
 Observe Time: 36 | S: 23519393.0, E: 0.0, A: 85.2, I: 377.9, D: 8.8, R: 1976439.0
 Time: 36 | new_e:7.9, cum_e:3805.5, new_i:16.3, cum_i:5259.9
 True State Time: 37 | S: 23465523.1, E: 15.5, A: 79.9, I: 347.0, D: 8.9, R: 2030025.7, Budget: 1407491.60, score:-17054.18
 Observe Time: 37 | S: 23465859.0, E: 0.0, A: 79.9, I: 347.0, D: 8.9, R: 2030025.0
 Time: 37 | new_e:6.7, cum_e:3812.2, new_i:14.7, cum_i:5274.6
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 38 | S: 23413740.4, E: 13.9, A: 73.6, I: 318.2, D: 8.9, R: 20818

 True State Time: 51 | S: 22767978.0, E: 42.5, A: 63.7, I: 212.8, D: 9.6, R: 2727693.4, Budget: 990492.96, score:-18099.08
 Observe Time: 51 | S: 22768554.0, E: 2.0, A: 63.7, I: 212.8, D: 9.6, R: 2727689.0
 Time: 51 | new_e:17.9, cum_e:4043.6, new_i:23.3, cum_i:5577.2
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 52 | S: 22724562.9, E: 40.9, A: 66.9, I: 209.5, D: 9.6, R: 2771110.1, Budget: 965712.31, score:-18175.11
 Observe Time: 52 | S: 22725165.0, E: 0.0, A: 66.9, I: 209.5, D: 9.6, R: 2771105.0
 Time: 52 | new_e:17.2, cum_e:4060.8, new_i:23.0, cum_i:5600.2
    model.name="unknown";
      - termination condition: infeasible
      - messag

 True State Time: 69 | S: 22111221.3, E: 217.9, A: 102.5, I: 413.6, D: 10.3, R: 3384034.4, Budget: 599353.78, score:-21269.37
 Observe Time: 69 | S: 22112783.0, E: 4.0, A: 102.5, I: 413.6, D: 10.3, R: 3384019.0
 Time: 69 | new_e:88.9, cum_e:4765.4, new_i:93.6, cum_i:6387.8
 True State Time: 70 | S: 22089014.5, E: 233.4, A: 127.1, I: 455.5, D: 10.3, R: 3406159.2, Budget: 583549.36, score:-21684.74
 Observe Time: 70 | S: 22090719.0, E: 0.0, A: 127.1, I: 455.5, D: 10.3, R: 3406138.0
 Time: 70 | new_e:95.1, cum_e:4860.5, new_i:102.0, cum_i:6489.8
 True State Time: 71 | S: 22064471.7, E: 270.1, A: 130.4, I: 500.5, D: 10.4, R: 3430616.9, Budget: 567637.71, score:-22164.67
 Observe Time: 71 | S: 22066333.0, E: 0.0, A: 130.4, I: 500.5, D: 10.4, R: 3430590.0
 Time: 71 | new_e:109.9, cum_e:4970.4, new_i:116.2, cum_i:6606.0
 True State Time: 72 | S: 22040115.7, E: 311.8, A: 137.5, I: 554.1, D: 10.4, R: 3454870.4, Budget: 553772.77, score:-22717.90
 Observe Time: 72 | S: 22042158.0, E: 0.0, A: 137

 True State Time: 81 | S: 21895516.9, E: 465.6, A: 634.4, I: 1441.8, D: 11.8, R: 3597929.6, Budget: 386043.15, score:-30778.64
 Observe Time: 81 | S: 21900281.0, E: 0.0, A: 634.4, I: 1441.8, D: 11.8, R: 3597770.0
 Time: 81 | new_e:227.0, cum_e:6935.8, new_i:269.2, cum_i:8797.9
pyomo fail
pyomo fail
pyomo fail
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
pyomo fail
pyomo fail
pyomo fail
 True State Time: 82 | S: 21883354.8, E: 594.8, A: 564.3, I: 1546.6, D: 12.1, R: 3609927.4, Budget: 349262.65, score:-31829.62
 Observe Time: 82 | S: 21888423.0, E: 37.0, A: 564.3, I: 1546.6, D: 12.1, R: 3609752.0
 Time: 82 | new_e:239.9, cum_e:7175.8, new_i:284.1, cum_i:9082.1
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasibl

In [7]:
(time.time() - tic) / ( T - start_time + 1)

173.0070998949163